<a href="https://colab.research.google.com/github/stephe-hu/testing/blob/main/hha507wk6_anova_analysis_python_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packaages

In [1]:
pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as multi

# 1. Data Preparation


## Research Question: How do the race of the patient (race) and their A1C test results (A1Cresult) jointly influence the duration of their hospital stay(time_in_hospital)?
### Factor 1 (Race - race): Caucasian, Asian, African American, Hispanic, Other
### Factor 2 (A1C test result - a1c): >8%, 7%, Normal, None (not measure)
### Dependent Variable: Duration hospital stay (time_in_hospital)


In [3]:
# fetch dataset
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)

# data (as pandas dataframes)
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets


/usr/local/lib/python3.10/dist-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [5]:
df = pd.DataFrame(X)
df

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,No
1,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,...,No,No,No,No,No,No,No,No,No,Yes
3,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,...,No,No,Steady,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101762,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,MC,NaN,...,No,No,Steady,No,No,No,No,No,No,Yes
101763,Caucasian,Male,[70-80),NaN,1,1,7,1,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101764,Caucasian,Female,[80-90),NaN,2,3,7,10,MC,Surgery-General,...,No,No,Up,No,No,No,No,No,Ch,Yes


In [ ]:
# Convert to CSV file
df.to_csv('diabetes.csv')


In [ ]:
# Drop rows with missing 'race' values
df = df.dropna(subset=['race'])

# Convert to CSV file
df.to_csv('clean.csv')

In [ ]:
# Create a subset of the data that includes only the variables of interest
subset = df[['time_in_hospital', 'race', 'A1Cresult']]
subset

,time_in_hospital,race,A1Cresult
0,1,Caucasian,None
1,3,Caucasian,None
2,2,AfricanAmerican,None
3,2,Caucasian,None
4,1,Caucasian,None
...,...,...,...
101761,3,AfricanAmerican,>8
101762,5,AfricanAmerican,None
101763,1,Caucasian,None
101764,10,Caucasian,None


In [ ]:
subset.dtypes

time_in_hospital     int64
race                object
A1Cresult           object
dtype: object

# 2. Assumption Checks


## Normality test

In [ ]:
print('Main DV: ', stats.shapiro(df['time_in_hospital']))

Main DV:  ShapiroResult(statistic=0.8867098093032837, pvalue=0.0)


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
groups = df.groupby(['race', 'A1Cresult'])

for (race_status, A1Cresult_status), group_df in groups:
    _, p_value = stats.shapiro(group_df['time_in_hospital'])

    print(f"Group ({race_status}, {A1Cresult_status}):")
    print(f"P-value from Shapiro-Wilk Test: {p_value}\n") # normally distributed if >0.05

Group (AfricanAmerican, >7):
P-value from Shapiro-Wilk Test: 1.8906119489850806e-16

Group (AfricanAmerican, >8):
P-value from Shapiro-Wilk Test: 6.7359146155303186e-34

Group (AfricanAmerican, None):
P-value from Shapiro-Wilk Test: 0.0

Group (AfricanAmerican, Norm):
P-value from Shapiro-Wilk Test: 2.9696700824519264e-25

Group (Asian, >7):
P-value from Shapiro-Wilk Test: 0.0023467366117984056

Group (Asian, >8):
P-value from Shapiro-Wilk Test: 2.233798113593366e-05

Group (Asian, None):
P-value from Shapiro-Wilk Test: 2.6558121925662052e-21

Group (Asian, Norm):
P-value from Shapiro-Wilk Test: 0.0007482930086553097

Group (Caucasian, >7):
P-value from Shapiro-Wilk Test: 1.7925673399754353e-38

Group (Caucasian, >8):
P-value from Shapiro-Wilk Test: 0.0

Group (Caucasian, None):
P-value from Shapiro-Wilk Test: 0.0

Group (Caucasian, Norm):
P-value from Shapiro-Wilk Test: 1.930148504761003e-41

Group (Hispanic, >7):
P-value from Shapiro-Wilk Test: 2.1721559733123286e-06

Group (Hispanic

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
# Levene's Test
stats.levene(
    df['time_in_hospital'][df['race'] == 'AfricanAmerican'][df['A1Cresult'] == '>7'],
    df['time_in_hospital'][df['race'] == 'AfricanAmerican'][df['A1Cresult'] == '>8'],
    df['time_in_hospital'][df['race'] == 'AfricanAmerican'][df['A1Cresult'] == 'Norm'],
    df['time_in_hospital'][df['race'] == 'AfricanAmerican'][df['A1Cresult'] == 'None'],
    df['time_in_hospital'][df['race'] == 'Asian'][df['A1Cresult'] == '>7'],
    df['time_in_hospital'][df['race'] == 'Asian'][df['A1Cresult'] == '>8'],
    df['time_in_hospital'][df['race'] == 'Asian'][df['A1Cresult'] == 'Norm'],
    df['time_in_hospital'][df['race'] == 'Asian'][df['A1Cresult'] == 'None'],
    df['time_in_hospital'][df['race'] == 'Caucasian'][df['A1Cresult'] == '>7'],
    df['time_in_hospital'][df['race'] == 'Caucasian'][df['A1Cresult'] == '>8'],
    df['time_in_hospital'][df['race'] == 'Caucasian'][df['A1Cresult'] == 'Norm'],
    df['time_in_hospital'][df['race'] == 'Caucasian'][df['A1Cresult'] == 'None'],
    df['time_in_hospital'][df['race'] == 'Hispanic'][df['A1Cresult'] == '>7'],
    df['time_in_hospital'][df['race'] == 'Hispanic'][df['A1Cresult'] == '>8'],
    df['time_in_hospital'][df['race'] == 'Hispanic'][df['A1Cresult'] == 'Norm'],
    df['time_in_hospital'][df['race'] == 'Hispanic'][df['A1Cresult'] == 'None'],
    df['time_in_hospital'][df['race'] == 'Other'][df['A1Cresult'] == '>7'],
    df['time_in_hospital'][df['race'] == 'Other'][df['A1Cresult'] == '>8'],
    df['time_in_hospital'][df['race'] == 'Other'][df['A1Cresult'] == 'Norm'],
    df['time_in_hospital'][df['race'] == 'Other'][df['A1Cresult'] == 'None'],
)

LeveneResult(statistic=7.4501020039258, pvalue=9.580811219790571e-21)

## Interpretation

### The shapiro test shows us that 20/20 combination have p-values <0.05, therefore they are NOT NORMALLY distributed.

### The Levene test shows us that the homogeneity of variance is not equal, with a levene test statistic of 7.45. Moreover, a p-value of 9.58e-21, which is less than 0.05, indicates that the variances are significantly different.

# 3. Conduct the ANOVA

In [ ]:
model = ols('time_in_hospital ~ C(race) * C(A1Cresult)', data=df).fit()

In [ ]:
# Performing the two-way ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

                             sum_sq       df           F        PR(>F)
C(race)                  641.735130      4.0   18.075566  7.501203e-15
C(A1Cresult)            3895.231358      3.0  146.287793  1.363867e-94
C(race):C(A1Cresult)     276.823623     12.0    2.599070  1.845652e-03
Residual              882895.150091  99473.0         NaN           NaN


## Interpretation

### The ANOVA result shows that both independent variables (race and A1C results) have a significant association with the dependent variable (hospital stay), and that there is a significant interaction between the independent variables.

# 4. Post-hoc test

In [ ]:
# Now, perform Tukey's HSD test
post_hoc = multi.MultiComparison(df['time_in_hospital'], df['race'])
tukey_result = post_hoc.tukeyhsd()
print(tukey_result)


      Multiple Comparison of Means - Tukey HSD, FWER=0.05       
     group1       group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------------
AfricanAmerican     Asian  -0.5125 0.0002 -0.8396 -0.1855   True
AfricanAmerican Caucasian  -0.1221    0.0 -0.1879 -0.0564   True
AfricanAmerican  Hispanic   -0.448    0.0 -0.6378 -0.2582   True
AfricanAmerican     Other  -0.2343 0.0278 -0.4523 -0.0163   True
          Asian Caucasian   0.3904 0.0087  0.0673  0.7135   True
          Asian  Hispanic   0.0646 0.9894 -0.3043  0.4335  False
          Asian     Other   0.2783 0.2778 -0.1059  0.6624  False
      Caucasian  Hispanic  -0.3258    0.0 -0.5087  -0.143   True
      Caucasian     Other  -0.1121 0.5995 -0.3241  0.0998  False
       Hispanic     Other   0.2137 0.2175 -0.0631  0.4905  False
----------------------------------------------------------------


## Interpretation:
### The difference in hospital stay between African American and Asian is statistically significant.
### The difference in hospital stay between African American and Caucasian is statistically significant.
### The difference in hospital stay between African American and Hispanic is statistically significant.
### The difference in hospital stay between African American and Other is statistically significant.
### The difference in hospital stay between Asian and Caucasian is statistically significant.
### The difference in hospital stay between Asian and Hispanic is not statistically significant.
### The difference in hospital stay between Asian and Other is not statistically significant.
### The difference in hospital stay between Caucasian and Hispanic is statistically significant.
### The difference in hospital stay between Caucasian and Other is not statistically significant.
### The difference in hospital stay between Hispanic and Other is not statistically significant.

In [ ]:
post_hoc = multi.MultiComparison(df['time_in_hospital'], df['A1Cresult'])
tukey_result = post_hoc.tukeyhsd()
print(tukey_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    >7     >8   -0.157 0.0397 -0.3089 -0.0051   True
    >7   None   -0.588    0.0 -0.7162 -0.4598   True
    >7   Norm   0.0238  0.983 -0.1425  0.1902  False
    >8   None   -0.431    0.0 -0.5209 -0.3412   True
    >8   Norm   0.1808 0.0046  0.0418  0.3198   True
  None   Norm   0.6118    0.0  0.4993  0.7243   True
----------------------------------------------------


## Interpretation
### The difference in hospital stay between patients with an A1C result of >7% and patients with an A1C result of >8% is statistically significant.
### The difference in hospital stay between patients with an A1C result of >7% and patients with an unmeasured A1C result is statistically significant.
### The difference in hospital stay between patients with an A1C result of >7% and patients with a normal A1C result is not statistically significant.
### The difference in hospital stay between patients with an A1C result of >8% and patients with an unmeasured A1C result is statistically significant.
### The difference in hospital stay between patients with an A1C result of >8% and patients with an unmeasured A1C result is statistically significant.
### The difference in hospital stay between patients with an unmeasured A1C result and patients with a normal A1C result is statistically significant.